In [ ]:
import requests
from pprint import pprint
import os
import subprocess
import boto3
from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient
import folium
from ipyleaflet import *
import rasterio
from rasterio.plot import show, show_hist
from matplotlib import pyplot
import datetime
from retrying import retry
import time

# while useful, the python bindings for gdal don't yet include
# its full functionality. Run gdal from the command line or
# use "!gdalinfo [args] (or whichever cli gdal command you like)
# to run commands from the notebook
from osgeo import gdal

## Genarating your API key


Radiant's platform uses a refresh-session token system to connect to the backend. Session tokens are ephemeral (they expire), so we can use a Swagger Client object to generate a persistent API key.

In [ ]:
 def generate_api_key(refresh_token):
    """
    Use the refresh token you created at
    https://app.radiant.earth/user/me/settings/api-tokens
    to generate an API token.
    """

    http_client = RequestsClient()

    spec_url = 'https://raw.githubusercontent.com/radiantearth/radiantearth-python-client/dev/radiantearth/spec.yml'

    config = {'validate_responses': False}
    client = SwaggerClient.from_url(spec_url, http_client=http_client, config=config)
    post_body = {'refresh_token': refresh_token}
    api_key = client.Authentication.post_tokens(
                    refreshToken=post_body).future.result().json()['id_token']
    
    return api_key

In [ ]:
api_key = generate_api_key('VGlFKlygOSMBEgYYouCy5RVuo3yq4ETUyXr3rHUSUe4gD')

base_url = "https://api.radiant.earth/platform/"

headers = {"Authorization": api_key,
          "Accept": "application/json"
          }

In [ ]:
!gdalinfo /workdir/HHNorthExport/HHNorthExport.tif